## Лабораторная работа №5 Синтаксис
#### Предмет: Методы интеллектуального анализа текстов
#### Студент: Леонова Алина, СБ № 1032212306, НФИмд-01-21

# 5. Синтаксис


1.	Объяснить значение трех UD-тегов синтаксических отношений на примере фрагмента размеченного корпуса.
2.	Написать функцию разбиения сложносочиненного предложения из двух частей на простые.
3.	Написать функцию нахождения наименьшего общего предка двух токенов в дереве зависимостей.
4.	Сравнить три пары предложений двумя методами: сравнением расстояния редактирования деревьев зависимостей (zss) и косинусной мерой между BERT-эмбеддингами.


### 5.4 Сравнить три пары предложений двумя методами: сравнением расстояния редактирования деревьев зависимостей и косинусной мерой между BERT-эмбеддингами

In [1]:
%%bash
pip install svgling git+https://github.com/IINemo/isanlp.git ufal.udpipe natasha
wget -q https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/russian-syntagrus-ud-2.5-191206.udpipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/IINemo/isanlp.git to /tmp/pip-req-build-k10g5d4t
  Created wheel for isanlp: filename=isanlp-0.0.7-py3-none-any.whl size=47867 sha256=1380141349d2d4493a601f09d35de1236db188945c461fd4e163a0afa5ba56d6
  Stored in directory: /tmp/pip-ephem-wheel-cache-lu3ftb52/wheels/15/b0/47/d8bfd7a8c44162856ecea49238bbe512d54ee3b0af191da75f
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp37-cp37m-linux_x86_64.whl size=5626619 sha256=6cf94e95b3a058f1ca7539037c7e0d56db86ca88e5b928bbd7e60afb499a97c0
  Stored in directory: /root/.cache/pip/wheels/b8/b5/8e/3da091629a21ce2d10bf90759d0cb034ba10a5cf7a01e83d64
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=af5da90b15a21694efc8c99243365d5c3541568ea79a0c1b019181357b667c60
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81c

  Running command git clone -q https://github.com/IINemo/isanlp.git /tmp/pip-req-build-k10g5d4t


In [2]:
examples = ["Привет, у нас на кухне нашли плесень!", 
            "На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.",
            "Привет, у них в подвале нашли клад!" ]

Построение стрелочек связей предложения

In [ ]:
from natasha.doc import DocToken, syntax_markup

def isanlp2natasha(annotation):
    result = []
    sentence_counter = 0
    token_counter = 0
    
    for i, token in enumerate(annotation['tokens']):
            
        token_pos = annotation['postag'][sentence_counter][token_counter]
        token_feats = list(annotation['morph'][sentence_counter][token_counter].values())
        token_syntax = annotation['syntax_dep_tree'][sentence_counter][token_counter]
        
        result.append(
            DocToken(
                start=token.begin, 
                stop=token.end, 
                text=token.text, 
                id=f"{sentence_counter}_{token_counter}", 
                head_id=f"{sentence_counter}_{token_syntax.parent}", 
                rel=token_syntax.link_name, 
                pos=token_pos, 
                feats=token_feats
            )
        )
        
        token_counter += 1
        
        if token_counter == len(annotation['syntax_dep_tree'][sentence_counter]):
            token_counter = 0
            sentence_counter += 1
            
        if sentence_counter == len(annotation['syntax_dep_tree']):
            return result

In [ ]:
from isanlp.processor_udpipe import ProcessorUDPipe
processor = ProcessorUDPipe('russian-syntagrus-ud-2.5-191206.udpipe')

for s in examples:
  res = processor(s)
  syntax_markup(isanlp2natasha(res)).print()

         Привет  
┌──────► ,       punct
│     ┌► у       case
│ ┌──►└─ нас     obl
│ │   ┌► на      case
│ │ ┌►└─ кухне   obl
└─└─└─┌─ нашли   
      └► плесень obj
         !       
        ┌──► На            case
        │ ┌► нашей         det
        └─└─ кухне         obl
┌───────└─┌─ нашли         
│   ┌─┌─┌─└► много         nsubj
│   │ │ └──► всего         obl
│   │ └────► :             punct
│ ┌─└──►┌─── бактерии      parataxis
│ │     │ ┌► ,             punct
│ │     └►└─ грибки        conj
│ │     ┌──► и             cc
│ │     │ ┌► позавчерашнее amod
│ └────►└─└─ молоко        conj
└──────────► .             punct
         Привет  
┌──────► ,       punct
│     ┌► у       case
│ ┌──►└─ них     obl
│ │   ┌► в       case
│ │ ┌►└─ подвале obl
└─└─└─┌─ нашли   
      └► клад    obj
         !       


#### Расстояние редактирования:

In [3]:
! pip install zss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for zss: filename=zss-1.2.0-py3-none-any.whl size=6747 sha256=acd6b9d3cec486cbc859ef6903e0164dea535beb394281e47969a08dcbcf3b61
  Stored in directory: /root/.cache/pip/wheels/df/92/2a/0f4fc18d986820ca2d4deabf2bd91d6a5893b926d717312520
Successfully built zss


In [4]:
# Морфосинтаксический анализатор UDPipe для русского языка

from isanlp.processor_udpipe import ProcessorUDPipe
processor = ProcessorUDPipe('russian-syntagrus-ud-2.5-191206.udpipe')

In [15]:
from zss import simple_distance, Node

def pos_dep_tree(postags, syntax_dep_tree):
    #Конвертируем результат морфосинтаксического анализа в zss-дерево из частей речи и синтаксических связей NOUN -> advmod -> VERB 
    root = Node('root')
    pos_nodes = {-1: root}
    for i, postag in enumerate(postags):
        pos_nodes[i] = Node(postag)

    for i, dependency_edge in enumerate(syntax_dep_tree):
        # <синтаксическое отношение, порядковый номер вершины (токена), порядковый номер его родительской вершины>
        relation, index, parent_index = dependency_edge['link_name'], i, dependency_edge['parent']
        print(relation, index, parent_index)
        
        relation_node = Node(relation)
        pos_nodes[parent_index].addkid(relation_node)
        relation_node.addkid(pos_nodes[index])
    print()
    return root


def sent_dep_tree(sent):
    #Получаем список постегов и список синтаксических аннотаций слов предложения, полученные анализатором
    result = processor(sent)
    postags = result['postag'][0]
    syntax_dep_tree = []
    for i in result['syntax_dep_tree'][0]:
      syntax_dep_tree.append(vars(i))

    return pos_dep_tree(postags, syntax_dep_tree)


def dep_tree_similarity(dep1, dep2, smoothing=5.0):
    return smoothing / (smoothing + simple_distance(dep1, dep2))

def sentence_similarity(sent1, sent2, smoothing=5.0):
    return dep_tree_similarity(sent_dep_tree(sent1), sent_dep_tree(sent2), smoothing)

In [17]:
for pair in [[0, 1], [1, 2], [0, 2]]:
    print('\n*pair', pair)
    print((examples[pair[0]], examples[pair[1]],
           sentence_similarity(examples[pair[0]], examples[pair[1]])))


*pair [0, 1]
root 0 -1
punct 1 6
case 2 3
obl 3 6
case 4 5
obl 5 6
conj 6 0
obj 7 6
punct 8 0

case 0 2
det 1 2
obl 2 3
root 3 -1
nsubj 4 3
obl 5 4
punct 6 4
parataxis 7 4
punct 8 9
conj 9 7
cc 10 12
amod 11 12
conj 12 7
punct 13 3

('Привет, у нас на кухне нашли плесень!', 'На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.', 0.05263157894736842)

*pair [1, 2]
case 0 2
det 1 2
obl 2 3
root 3 -1
nsubj 4 3
obl 5 4
punct 6 4
parataxis 7 4
punct 8 9
conj 9 7
cc 10 12
amod 11 12
conj 12 7
punct 13 3

root 0 -1
punct 1 6
case 2 3
obl 3 6
case 4 5
obl 5 6
conj 6 0
obj 7 6
punct 8 0

('На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.', 'Привет, у них в подвале нашли клад!', 0.05263157894736842)

*pair [0, 2]
root 0 -1
punct 1 6
case 2 3
obl 3 6
case 4 5
obl 5 6
conj 6 0
obj 7 6
punct 8 0

root 0 -1
punct 1 6
case 2 3
obl 3 6
case 4 5
obl 5 6
conj 6 0
obj 7 6
punct 8 0

('Привет, у нас на кухне нашли плесень!', 'Привет, у них в подвале нашли к

Пара предложений [0,2] синтаксически одинаковы (количество слов и вычисленные для них синтаксические отношения совпадают) и значение их сходства ранво 1. Синтаксическая схожесть пар предложений [0,1] и [1,2] получилась равной лишь 0.053.

#### Контекстные эмбеддинги

In [18]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 30.9 MB/s 
     |████████████████████████████████| 6.6 MB 71.3 MB/s 
     |████████████████████████████████| 163 kB 78.6 MB/s 


In [19]:
from transformers import BertTokenizer, BertModel
import torch

# Использую sentence-BERT, дообученный на задаче языкового вывода
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/711M [00:00<?, ?B/s]

In [124]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

# Создание вектора из предложения
def embed_sentence(sentence: str):
    # Токенизация и предсказания модели
    input_ids = tokenizer(sentence, return_tensors="pt")
    outputs = model(**input_ids)
    
    r = torch.mean(outputs[0].detach(), axis=0).numpy()   # массив векторов слов
    return np.concatenate(r)  # один вектор на всё предложение


# Определение косинусной близости двух предложений 
def cos_similarity(sent1, sent2):
  v_s1 = embed_sentence(sent1)
  v_s2 = embed_sentence(sent2)

  sh1 = v_s1.shape[0]
  sh2 = v_s2.shape[0]

  # предложения часто разной длины, добавляю к меньшему нули до размера большего  
  print(sh1, sh2)
  if sh1 > sh2:
    v_s2 = np.append(v_s2, np.zeros(sh1 - sh2))
  elif sh1 < sh2:
    v_s1 = np.append(v_s1, np.zeros(sh2 - sh1))
  print(v_s1.shape[0], v_s2.shape[0])
      
  # косинусная близость
  cos_sim = cosine_similarity(v_s1.reshape(1,-1), v_s2.reshape(1,-1))[0][0]
  return cos_sim

In [125]:
for pair in [[0, 1], [1, 2], [0, 2]]:
    print('\n*pair', pair)
    print((examples[pair[0]], examples[pair[1]],
           cos_similarity(examples[pair[0]], examples[pair[1]])))


*pair [0, 1]
9216 16128
16128 16128
('Привет, у нас на кухне нашли плесень!', 'На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.', 0.5646608777229581)

*pair [1, 2]
16128 8448
16128 16128
('На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.', 'Привет, у них в подвале нашли клад!', 0.37503175896986896)

*pair [0, 2]
9216 8448
9216 9216
('Привет, у нас на кухне нашли плесень!', 'Привет, у них в подвале нашли клад!', 0.6106255878578744)


Полученные косинусные близости показали, что предложения пары [0,2] оказались контекстно ближе друг к другу (0.61), чем остальные. Дальше идёт пара [0,1] cо схожестью в 0.56, наиболее непохожи предложения пары [1,2], схожесть - 0.375. 